In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

Codeforces Practice

In [4]:
def pair_of_numbers(N: int, nums: list[int]) -> tuple[int, int, list[int]]:
    """
    Starting at index, check how far right you can go. Repeat same step on the leftwards direction.
    Combine the left and right to get the actual window that index i can cover (where nums[i] is the gcd and divides all)

    Problem occurs where there are duplicates where we skip those numbers since it was already covered in the window. Backward
    pass travels in the opp direction and we might end up missing the proper range.

    For eg:   2, 3, 1, 1, 3
    Forward:  0  0  3  0  0
    Backward: 0  0  0  4  0

    We could choose to ignore duplicates but that would miss nonoverlapping ranges
    We could try merging intervals but it wouldn't be trivial and we would need to merge only cases where the values match (need to lookback the entire length making it N^2)

    Simpler approach is to store the duplicates into an array and fill it in when we update i.
    """

    # Forward pass
    forward: dict[int, int] = {i: i for i in range(N)}
    i = j = 0
    match: list[int] = []
    while j <= N:
        if j < N and nums[j] % nums[i] == 0:
            if nums[j] == nums[i]:
                match.append(j)
            forward[i], j = j, j + 1
        else:
            for k in match:
                forward[k] = forward[i]
            i, j, match = j, j + 1, []

    # Backward pass
    backward: dict[int, int] = {i: i for i in range(N)}
    i = j = N - 1
    match = []
    while j >= -1:
        if j >= 0 and nums[j] % nums[i] == 0:
            if nums[j] == nums[i]:
                match.append(j)
            backward[i], j = j, j - 1
        else:
            for k in match:
                backward[k] = backward[i]
            i, j, match = j, j - 1, []

    # Merge both passes and store to a map seperated by distance
    valid_pairs: collections.defaultdict[int, set[int]] = collections.defaultdict(set)
    max_: int = 0
    for i in range(N):
        diff = forward[i] - backward[i]
        valid_pairs[diff].add(backward[i] + 1)
        max_ = max(max_, diff)

    return len(valid_pairs[max_]), max_, sorted(valid_pairs[max_])

# Testing the solution
assert pair_of_numbers(9, [2,4,3,6,9,8,10,2,4]) == (1, 3, [6])
assert pair_of_numbers(5, [4,6,9,3,6]) == (1, 3, [2])
assert pair_of_numbers(5, [1,3,5,7,9]) == (1, 4, [1])
assert pair_of_numbers(5, [2,3,5,7,11]) == (5, 0, [1,2,3,4,5])
assert pair_of_numbers(5, [1,2,3,2,1]) == (1, 4, [1])
assert pair_of_numbers(5, [2,4,1,1,2]) == (1, 4, [1])

# Good substrings - 271D
class Trie:
    def __init__(self, end: bool = False) -> None:
        self.nodes: dict[str, 'Trie'] = dict()
        self.end = end

    def __getitem__(self, ch: str) -> 'Trie':
        self.nodes[ch] = self.nodes.get(ch, Trie())
        return self.nodes[ch]

def good_substrings(S: str, good_chars_: str, K: int) -> int:
    N, count = len(S), 0
    good_chars: set[str] = {chr(ord('a') + i) for i in range(26) if good_chars_[i] == '1'}
    root: Trie = Trie()
    for i in range(N):
        k_, curr = K, root
        for j in range(i, N):
            if S[j] not in good_chars:
                k_ -= 1
            if k_ >= 0:
                curr = curr[S[j]]
                count += not curr.end
                curr.end = True
            else:
                break

    return count

# Testing the solution
assert good_substrings("ababab", "01000000000000000000000000", 1) == 5
assert good_substrings("acbacbacaa", "00000000000000000000000000", 2) == 8

LC POTD

In [5]:
# https://leetcode.com/problems/create-binary-tree-from-descriptions/submissions/1321920493/?envType=daily-question&envId=2024-07-15
def createBinaryTree(descriptions: list[list[int]]) -> BinaryTreeNode:
    """Node without parent is the root"""
    # Create the binary tree
    nodes_dict: dict[int, BinaryTreeNode] = dict()
    parents: set[int] = set()
    children: set[int] = set()
    for parent, child, direction in descriptions:
        p_node, c_node = nodes_dict.get(parent, BinaryTreeNode(parent)), nodes_dict.get(child, BinaryTreeNode(child))
        nodes_dict[parent], nodes_dict[child] = p_node, c_node

        if direction == 1:
            p_node.left = c_node
        else:
            p_node.right = c_node

        if parent not in children:
            parents.add(parent)
        if child in parents:
            parents.remove(child)
        children.add(child)

    assert len(parents) == 1
    return nodes_dict[parents.pop()]

# Testing the solution
assert createBinaryTree([[20,15,1],[20,17,0],[50,20,1],[50,80,0],[80,19,1]]).to_list() == [50,20,80,15,17,19]
assert createBinaryTree([[1,2,1],[2,3,0],[3,4,1]]).to_list() == [1,2,None,None,3,4]

KMP Algorithm

In [6]:
def create_lps_array(pattern: str) -> list[int]:
    N = len(pattern)
    pi: list[int] = [0] * N
    j = 0
    for i in range(1, N):
        while pattern[j] != pattern[i] and j > 0:
            j = pi[j - 1]
        if pattern[i] == pattern[j]:
            j += 1
        pi[i] = j

    return pi

# Testing the solution
assert create_lps_array("abcdabc") == [0, 0, 0, 0, 1, 2, 3]

Codeforces Practice

In [7]:
def longest_prefix_suffix_palindome(S: str) -> str:
    def KMP(pattern: str) -> list[int]:
        pattern_length = len(pattern)
        pi: list[int] = [0] * pattern_length
        j = 0
        for i in range(1, pattern_length):
            while pattern[i] != pattern[j] and j > 0:
                j = pi[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            pi[i] = j

        return pi

    N = len(S)
    i, j = 0, N - 1

    # Find greatest k such that S[0..k] + S[N - k - 1.. N - 1] is a palindrome
    k = 0
    for k in range(N // 2):
        if S[k] != S[N - k - 1]:
            break

    # Find longest prefix-suffix of S[k..N-k-1]
    middle = S[k: N - k]
    lps = middle[:KMP(middle + '#' + middle[::-1])[-1]]

    # Find longest prefix-suffix of reversed(S[k..N-k-1])
    middle_reversed = middle[::-1]
    lps_reversed = middle_reversed[:KMP(middle_reversed + '#' + middle_reversed[::-1])[-1]]

    return S[:k] + (lps if len(lps) > len(lps_reversed) else lps_reversed[::-1]) + S[N - k:]

# Testing the solution
assert len(longest_prefix_suffix_palindome("abcdfdcecba")) == len("abcdfdcba")
assert len(longest_prefix_suffix_palindome("abbaxyzyx")) == len("xyzyx")
assert len(longest_prefix_suffix_palindome("acbba")) == len("abba")
assert len(longest_prefix_suffix_palindome("a")) == len("a")
assert len(longest_prefix_suffix_palindome("codeforces")) == len("c")

#### LC Practice - Strings - KMP

In [8]:
# LC: https://leetcode.com/problems/find-the-index-of-the-first-occurrence-in-a-string/
def strStr(haystack: str, needle: str) -> int:
    def create_lps(pattern: str, M: int) -> list[int]:
        j = 0
        lps = [0] * M
        for i in range(1, M):
            while pattern[i] != pattern[j] and j > 0:
                j = lps[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            lps[i] = j

        return lps

    N, M = len(haystack), len(needle)
    lps = create_lps(needle, M)
    j = 0
    for i in range(N):
        while haystack[i] != needle[j] and j > 0:
            j = lps[j - 1]
        if haystack[i] == needle[j]:
            j += 1
        if j >= M:
            return i - j + 1

    return -1

# Testing the solution
assert strStr("asabadzabababababaaababaabba", "abababab") == 7

In [9]:
# https://leetcode.com/problems/repeated-substring-pattern/
def repeatedSubstringPattern(S: str) -> bool:
    def create_lps(pattern: str) -> list[int]:
        pattern_length, j = len(pattern), 0
        lps: list[int] = [0] * pattern_length
        for i in range(1, pattern_length):
            while pattern[i] != pattern[j] and j > 0:
                j = lps[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            lps[i] = j

        return lps

    # Create LPS, find the longest repeating pattern length
    # Ensure that the pattern repeats itself
    N = len(S)
    lps = create_lps(S)
    repeat_str_length = N - lps[-1]

    for i in range(N):
        if S[i % repeat_str_length] != S[i]:
            return False
    return (repeat_str_length < N) and (N % repeat_str_length == 0)

# Testing the solution
assert repeatedSubstringPattern("acbaaacbaa") == repeatedSubstringPattern("aaabbaaabb") == True
assert repeatedSubstringPattern("abcabcabc") == repeatedSubstringPattern("abaababaab") == True
assert repeatedSubstringPattern("abcabcabz") == repeatedSubstringPattern("aba") == False

In [10]:
# TLE :(
def watto_mechanism_trie(input_patterns: list[str], search_patterns: list[str]) -> list[bool]:
    class TrieNode:
        def __init__(self, end: bool = False):
            self.nodes: collections.defaultdict[str, 'TrieNode'] = collections.defaultdict(TrieNode)
            self.end: bool = end

        def __getitem__(self, ch: str) -> 'TrieNode':
            return self.nodes[ch]

    def search(string: str) -> bool:
        N = len(string)
        stack: list[tuple[TrieNode, int, bool]] = [(ROOT, 0, True)]
        while stack:
            curr, idx, editable = stack.pop()
            if idx == N:
                if not editable and curr.end:
                    return True
            else:
                for next_ch in ['a', 'b', 'c']:
                    if next_ch in curr.nodes:
                        if next_ch == string[idx]:
                            stack.append((curr[next_ch], idx + 1, editable))
                        elif next_ch != string[idx] and editable:
                            stack.append((curr[next_ch], idx + 1, False))
        return False

    # Convert the input patterns into a trie data strucute
    ROOT = TrieNode()
    for inp in input_patterns:
        curr = ROOT
        for ch in inp:
            curr = curr[ch]
        curr.end = True

    results: list[bool] = []
    for search_pattern in search_patterns:
        results.append(search(search_pattern))

    return results

# Testing the solution
assert watto_mechanism_trie(["aaaaa", "acacaca"], ["aabaa", "ccacacc", "caaac"]) == [True, False, False]
assert watto_mechanism_trie(["acbacbacb"], ["cbacbacb", "acbacbac", "aacbacbacb", "acbacbacbb", "acbaabacb"]) == [False, False, False, False, True]
assert watto_mechanism_trie(["ab", "cacab", "cbabc", "acc", "cacab"], ["abc", "aa", "acbca", "cb"]) == [True, True, False, True]

In [11]:
def watto_mechanism(input_patterns: list[str], search_patterns: list[str]) -> list[bool]:
    """
    Idea from editorial:
        1. Compute and store the polynomial rolling hashes for each of the input_patterns
        2. For each search string, compute possibilities and check if they exist in the hash
    """

    # Constants for polynomial rolling hash
    P, M = 257, 2 ** 61 + 7

    def poly_hash(string: str) -> int:
        nonlocal P, M
        N, hash_val, p_ = len(string), 0, 1
        for i in range(N):
            hash_val = (hash_val + (ord(string[i]) * p_)) % M
            p_ = (p_ * P) % M
        return hash_val

    def search(string: str) -> bool:
        nonlocal P, M
        N, hash_val, p_ = len(string), poly_hash(string), 1
        for i in range(N):
            for ch in ('a', 'b', 'c'):
                modified_hash = (hash_val + (ord(ch) - ord(string[i])) * p_) % M
                if ch != string[i] and modified_hash in hashes:
                    return True
            p_ = (p_ * P) % M
        return False

    # Store all the input strs as poly hashes
    hashes: set[int] = set()
    for inp in input_patterns:
        hashes.add(poly_hash(inp))

    # Iterate through the search strings, compute all hashes for all combinations in O(1) time
    results: list[bool] = []
    for inp in search_patterns:
        results.append(search(inp))

    return results

# Testing the solution
assert watto_mechanism(["aaaaa", "acacaca"], ["aabaa", "ccacacc", "caaac"]) == [True, False, False]
assert watto_mechanism(["acbacbacb"], ["cbacbacb", "acbacbac", "aacbacbacb", "acbacbacbb", "acbaabacb"]) == [False, False, False, False, True]
assert watto_mechanism(["ab", "cacab", "cbabc", "acc", "cacab"], ["abc", "aa", "acbca", "cb"]) == [True, True, False, True]

Codeforces Practice

In [12]:
def palindrome_degree(string: str) -> int:
    """
    Iterate through string, check if current substr is a palindrome.
    If it is, degree equals degree[substr // 2] + 1.
    Assume degree[substr // 2] if not found as 0.
    Return sum of all degrees.

    Checking if substr is a palindrome, check if prefix_hash hash(S[i..j]) == rev_prefix_hash hash(S[j..i])

    PREFIX_HASH = sum(ord(s[i]) * p ** i)
        abc: a + bP + cP^2

    REV_PREFIX_HASH is a little tricky but when running in a loop it is pretty simple
        abc: (rev_prefix_hash * P) + ord(s[i])
            a -> a
            b -> aP + b
            c -> aP^2 + bP + c
    """
    # Paramaters for rolling hash
    P, M = 257, 2**61 - 1

    N = len(string)
    degree: list[int] = [0 for i in range(N)]
    prefix_hash = rev_prefix_hash = ord(string[0])
    total_degree = degree[0] = 1
    p = P
    for i in range(1, N):
        prefix_hash, p = (prefix_hash + ord(string[i]) * p) % M, (P * p) % M
        rev_prefix_hash = (rev_prefix_hash * P + ord(string[i])) % M
        if prefix_hash == rev_prefix_hash:
            degree[i] = degree[(i - 1) // 2] + 1
            total_degree += degree[i]

    return total_degree

# Testing the solution
assert palindrome_degree("a2A") == 1
assert palindrome_degree("abacaba") == 6

In [13]:
def spy_syndrome2(secret: str, words: list[str]) -> str:
    # Parameters for rolling Hash
    P, M = 257, 10**9 + 7

    def poly_hash(string: str) -> int:
        nonlocal P, M
        word_length, hash_value, p = len(string), 0, 1
        for i in range(word_length):
            hash_value, p = (hash_value + p * ord(string[i].lower())) % M, (p * P) % M

        return hash_value

    # Store the list of words as hashes
    hashes: dict[int, str] = dict()
    for word in words:
        hashes[poly_hash(word)] = word

    # Compute the rev prefix hash as we iterate through the secret, if
    # match found reset and add to list of words found, note that we
    # might need to backtrack which is handled by the stack code here!
    result: list[str] = []
    secret_length = len(secret)
    stack: list[tuple[int, int]] = [(0, 0)]
    while stack:
        idx, hash_value = stack.pop()
        if idx == secret_length:
            if hash_value == 0:
                break
            else:
                result.pop()
        else:
            hash_value = (hash_value * P + ord(secret[idx].lower())) % M
            stack.append((idx + 1, hash_value))
            if hash_value in hashes:
                result.append(hashes[hash_value])
                stack.append((idx + 1, 0))

    return ' '.join(result)

# Testing the solution
assert spy_syndrome2("ariksihsidlihcdnaehsetahgnisol", ["Kira", "hates", "is", "he", "losing", "death", "childish", "L", "and", "Note"]) == "Kira is childish and he hates losing"
assert spy_syndrome2("iherehtolleh", ["HI", "Ho", "there", "HeLLo", "hello"]).lower() == "HI there HeLLo".lower()
assert spy_syndrome2("ababaaba", ["aba", "ababa"]) == "ababa aba"

Codeforces Practice

In [14]:
def xor_paths(M: int, N: int, matrix: list[list[int]], K: int) -> int:
    """
    Meet in the middle to reduce the time complexity.
    """

    valid_path_count, max_moves = 0, M + N - 2
    counts: collections.defaultdict[tuple[int, int, int], int] = collections.defaultdict(int)

    # Starting from (0, 0), make max_moves // 2: travelling right or down
    stack: list[tuple[int, int, int, int]] = [(0, 0, 0, 0)]
    while stack:
        i, j, xor_sum, moves = stack.pop()
        xor_sum ^= matrix[i][j]
        if moves == max_moves // 2:
            counts[i, j, xor_sum] += 1
        else:
            if i + 1 < M:
                stack.append((i + 1, j, xor_sum, moves + 1))
            if j + 1 < N:
                stack.append((i, j + 1, xor_sum, moves + 1))

    # Starting from (M - 1, N - 1), make max_moves - (max_moves // 2): travelling left or up
    stack = [(M - 1, N - 1, K, 0)]
    while stack:
        i, j, xor_sum, moves = stack.pop()
        if moves == max_moves - (max_moves // 2):
            # In the end counts[i, j, xor_sum] that we reached from the
            # front and the end would be such that xor_sum ^ K == 0
            valid_path_count += counts[i, j, xor_sum]
        else:
            if i - 1 >= 0:
                stack.append((i - 1, j, xor_sum ^ matrix[i][j], moves + 1))
            if j - 1 >= 0:
                stack.append((i, j - 1, xor_sum ^ matrix[i][j], moves + 1))

    return valid_path_count

# Testing the solution
assert xor_paths(3, 3, [[2, 1, 5], [7, 10, 0], [12, 6, 4]], 11) == 3
assert xor_paths(3, 4, [[1, 3, 3, 3], [0, 3, 3, 2], [3, 0, 1, 1]], 2) == 5
assert xor_paths(3, 4, [[1, 3, 3, 3], [0, 3, 3, 2], [3, 0, 1, 1]], 1000000000000000000) == 0

CP Easy: Trees

In [15]:
def subordinates(n_employees: int, bosses: list[int]) -> list[int]:
    """
    The recusive solution using cache is trivial, using the
    iterative approach to prevent stackoverflow
    """

    subordinate_counts: list[int] = [-1 for i in range(n_employees + 1)]
    subordinates: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for i, boss in enumerate(bosses, start=2):
        subordinates[boss].append(i)

    # Starting with the director, count subordinates recursively using stack
    stack: list[int] = [1]
    while stack:
        emp, subordinate_count = stack[-1], 0
        for subordinate in subordinates[emp]:
            if subordinate_counts[subordinate] == -1:
                stack.append(subordinate)
            else:
                subordinate_count += 1 + subordinate_counts[subordinate]
        if emp == stack[-1]:
            subordinate_counts[emp] = subordinate_count
            stack.pop()

    return subordinate_counts[1:]

# Testing the solution
assert subordinates(5, [1, 1, 2, 3]) == [4, 1, 1, 0, 0]
assert subordinates(10, [1,1,1,1,1,1,1,1,1]) == [9,0,0,0,0,0,0,0,0,0]
assert subordinates(10, [1,2,3,4,5,6,7,8,9]) == [9,8,7,6,5,4,3,2,1,0]

In [16]:
def tree_matching(E: int, edges: list[tuple[int, int]]) -> int:
    """
    Only having a superficial understanding, converted with trial and
    errors the solution from recursive to iterative.
    """
    adjl: collections.defaultdict[int, list] = collections.defaultdict(list)
    for i, j in edges:
        adjl[i].append(j)
        adjl[j].append(i)

    # greedily keep matching a leaf with the only vertex adjacent to it while possible
    visited: set[int] = {1}
    picked: set[int] = set()
    stack: list[tuple[int, int]] = [(1, -1)]
    matching: int = 0
    while stack:
        curr, prev = stack[-1]
        for next_ in adjl[curr]:
            if next_ not in visited:
                visited.add(next_)
                stack.append((next_, curr))
        if stack[-1] == (curr, prev):
            stack.pop()
            # At the end of each recursive call, as per recursive code logic
            # we skip the first 'edge' (1, -1)
            if prev != -1 and prev not in picked and curr not in picked:
                matching += 1
                picked.add(curr)
                picked.add(prev)

    return matching

# Testing the solution
assert tree_matching(5, [(1,2), (1,3), (3,4), (3,5)]) == 2
assert tree_matching(10, [(8,9), (10,9), (1,4), (7,1), (8,3), (10,5), (4,2), (3,7), (6,4)]) == 4

Diameter of a tree:
Naive approach is to run DFS on all nodes which would lead to a time complexity of O(n * n).<br>
Better approach would be to run DFS from any node and find the farthest point.
From this point run DFS again and fnd the farthest point.
Compute distance between these two nodes.

In [17]:
def circumference(N: int, edges: list[tuple[int, int]]) -> int:
    def farthest(node: int) -> tuple[int, int]:
        result: tuple[int, int] = (-1, -1)
        queue: collections.deque[tuple[int, int, int]] = collections.deque([(node, -1, 0)])
        while queue:
            curr, prev, dist = queue.popleft()
            if dist > result[1]:
                result = curr, dist
            for next_ in adjl[curr]:
                if next_ != prev:
                    queue.append((next_, curr, dist + 1))

        return result

    # Create an adj list from the provided edges
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n1].append(n2)
        adjl[n2].append(n1)

    # Take any random node and run DFS to find the farthest point
    n1, _ = farthest(edges[0][0])
    n2, D = farthest(n1)

    # Wrong formula but this is the expected formula
    return 3 * D

# Testing the solution
assert circumference(3, [(3,2), (2,1)]) == 6
assert circumference(5, [(4,2), (1,4), (5,4), (3,4)]) == 6

LC Biweekly - 20th July 2024

In [18]:
def losingPlayer(x: int, y: int) -> str:
    alice_turn = True
    x_req, y_req = 1, 4
    while x >= x_req and y >= y_req:
        x, y = x - x_req, y - y_req
        alice_turn = not alice_turn

    return "Alice" if not alice_turn else "Bob"

# Testing the solution
assert losingPlayer(2, 7) == "Alice"
assert losingPlayer(4, 11) == "Bob"

In [19]:
def minimumLength(s: str) -> int:
    freq: collections.Counter = collections.Counter(s)
    return sum(map(lambda x: x if x <= 2 else 1 if x % 2 == 1 else 2, freq.values()))

# Testing the solution
assert minimumLength("abaacbcbb") == 5
assert minimumLength("aa") == 2

In [20]:
# TLE :)
def minChanges(nums: list[int], K: int) -> int:
    N, min_changes = len(nums), math.inf
    max_diff = max([*nums, K])
    for x in range(max_diff + 1):
        changes = 0.
        for i in range(N // 2):
            L, R = nums[i], nums[N - i - 1]
            if abs(L - R) == x:
                changes += 0
            elif (L - K <= x <= max(L, abs(L - K))) or (R - K <= x <= max(R, abs(R - K))):
                changes += 1
            elif 0 <= x <= K:
                changes += 2
            else:
                changes += math.inf

        min_changes = min(min_changes, changes)

    return int(min_changes)

# Testing the solution
assert minChanges([1,0,1,2,4,3], 4) == 2
assert minChanges([0,1,2,3,3,6,5,4], 6) == 2
assert minChanges([3,1,7,7,8,7,0,5,8,0,6,7,0,2,6,6], 8) == 6

LC Weekly Contest - 21st July 2024

In [21]:
def minBitChanges(N: int, K: int) -> int:
    changes = 0.
    while N > 0 or K > 0:
        N_bit, K_bit = N & 1, K & 1
        N, K = N >> 1, K >> 1
        if N_bit == 1 and K_bit == 0:
            changes += 1
        elif N_bit == K_bit:
            changes += 0
        else:
            changes += math.inf

    return int(changes) if not math.isinf(changes) else -1

# Testing the solution
assert minBitChanges(13, 4) == 2
assert minBitChanges(21, 21) == 0
assert minBitChanges(14, 13) == -1

In [22]:
def doesAliceWin(s: str) -> bool:
    vc = 0
    for ch in s:
        if ch in ('a', 'e', 'i', 'o', 'u'):
            vc += 1

    return vc != 0

# Testing the solution
assert doesAliceWin("leetcoder") == True
assert doesAliceWin("bcd") == False

In [23]:
def maxOperations(S: str) -> int:
    N = len(S)
    ops = ones = 0
    for i in range(N):
        if S[i] == '1':
            ones += 1
        elif S[i - 1] == '1':
            ops += ones

    return ops

# Testing the solution
assert maxOperations("1001101") == 4
assert maxOperations("00111") == 0
assert maxOperations("1000") == 1
assert maxOperations("1110") == 3

In [24]:
# Solved at 9:40 :)
def minimumOperations(nums: list[int], target: list[int]) -> int:
    def equal_sign(n1: int, n2: int) -> bool:
        return (n1 < 0 and n2 < 0) or (n1 > 0 and n2 > 0) or (n1 == n2 == 0)

    def split(left: int, right: int) -> list[tuple[int, int]]:
        result: list[tuple[int, int]] = []
        prev = left
        for i in range(left, right + 2):
            if i == right + 1 or not equal_sign(deltas[i], deltas[prev]):
                result.append((prev, i - 1))
                prev = i

        return result

    def count(left: int, right: int) -> int:
        min_ = abs(deltas[left])
        for i in range(left, right+1):
            deltas[i] = abs(deltas[i])
            min_ = min(min_, deltas[i])

        for i in range(left, right+1):
            deltas[i] -= min_

        slices: list[tuple[int, int]] = split(left, right)
        ops_ = min_
        if min_ > 0:
            for i, j in slices:
                ops_ += count(i, j)

        return ops_

    N, ops = len(nums), 0
    deltas: list[int] = [nums[i] - target[i] for i in range(N)]
    slices: list[tuple[int, int]] = split(0, N - 1)
    for i, j in slices:
        ops += count(i, j)

    return ops

# Testing the solution
assert minimumOperations([3,5,1,2], [4,6,2,4]) == 2
assert minimumOperations([1,3,2], [2,1,4]) == 5

In [25]:
def dynamic_dyameter(N: int, edges: list[tuple[int, int]]) -> list[int]:
    def BFS(start: int) -> tuple[int, set[int]]:
        distances: collections.defaultdict[int, set[int]] = collections.defaultdict(set)
        queue: collections.deque[tuple[int, int, int]] = collections.deque([(start, 0, 0)])
        max_dist = 0
        while queue:
            curr, prev, dist = queue.popleft()
            distances[dist].add(curr)
            max_dist = max(dist, max_dist)
            for next_ in adjl[curr]:
                if next_ != prev:
                    queue.append((next_, curr, dist + 1))

        return max_dist, distances[max_dist]

    # Convert edges to adjancency list
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n1].append(n2)
        adjl[n2].append(n1)

    # Find the farthest from a random node
    nodes = BFS(1)[1]
    leaves: set[int] = {*nodes}
    D, farthest_nodes = BFS(next(iter(leaves)))
    leaves.update(farthest_nodes)

    return [D + 1 if i in leaves else D for i in range(1, N + 1)]

# Testing the solution
assert dynamic_dyameter(3, [(3,2), (2,1)]) == [3,2,3]
assert dynamic_dyameter(5, [(4,2), (1,4), (5,4), (3,4)]) == [3,3,3,2,3]

In [26]:
def military_problem(n_officers: int, superiors: list[int], queries: list[tuple[int, int]]) -> list[int]:
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for officer, superior in enumerate(superiors, start=2):
        adjl[superior].append(officer)

    # DFS starting from the supreme commander - 1
    # Create variables for storing the DFS order and the start/end index for each
    order: list[int] = []
    ranges: collections.defaultdict[int, tuple[int, int]] = collections.defaultdict(lambda: (0, 0))
    stack: list[tuple[int, int]] = [(1, 0)]
    while stack:
        curr, idx = stack[-1]
        if idx == 0:
            order.append(curr)
            ranges[curr] = len(order) - 1, len(order) - 1
        if idx < len(adjl[curr]):
            stack[-1] = curr, idx + 1
            stack.append((adjl[curr][idx], 0))
        else:
            stack.pop()
            ranges[curr] = ranges[curr][0], len(order) - 1

    # Answer the queries
    result: list[int] = []
    for officer, kth in queries:
        start, end = ranges[officer]
        if end - start + 1 >= kth:
            result.append(order[start + kth - 1])
        else:
            result.append(-1)

    return result

# Testing the solution
assert military_problem(9, [1,2,2,3,5,3,6,7], [(1,5), (7,3), (1,8), (1,9)]) == [6,-1,9,4]
assert military_problem(9, [1,1,1,3,5,3,5,7], [(3,1), (1,5), (3,4), (7,3), (1,8), (1,9)]) == [3,6,8,-1,9,4]

Binary Lifting - LCA

In [27]:
# https://leetcode.com/problems/kth-ancestor-of-a-tree-node://leetcode.com/problems/kth-ancestor-of-a-tree-node/
class TreeAncestor:
    def __init__(self, N: int, parent: list[int]):
        # Store the ancestors into a sparse table
        self.ancestors: dict[int, collections.defaultdict[int, int]] = {}

        # Store for level 1 (2 ** 0 => 1)
        pending: list[int] = []
        level = 0
        for node in range(N):
            self.ancestors[node] = collections.defaultdict(lambda: -1)
            self.ancestors[node][level] = parent[node]
            if self.ancestors[node][level] != -1:
                pending.append(node)

        # Compute next level of ancestors until we hit -1 for all nodes
        level += 1
        while pending:
            still_pending: list[int] = []
            for node in pending:
                self.ancestors[node][level] = self.ancestors[self.ancestors[node][level - 1]][level - 1]
                if self.ancestors[node][level] != -1:
                    still_pending.append(node)
            pending, level = still_pending, level + 1

    def getKthAncestor(self, node: int, K: int) -> int:
        while K > 0 and node != -1:
            jump = int(math.log2(K))
            node = self.ancestors[node][jump]
            K -= 1 << jump # k - (2 ** jump)

        return node

# Testing the solution
tree_ancestor = TreeAncestor(7, [-1, 0, 0, 1, 1, 2, 2])
assert tree_ancestor.getKthAncestor(5, 2) == 0
assert tree_ancestor.getKthAncestor(6, 3) == -1
assert tree_ancestor.getKthAncestor(6, 0) == 6
assert tree_ancestor.getKthAncestor(6, 9) == -1

CP Practice - LCA

In [28]:
def sloth_naptime(N: int, edges: list[tuple[int, int]], Q: int, queries: list[tuple[int, int, int]]) -> list[int]:
    def get_distances_from_lca(N1: int, N2: int) -> tuple[int, int]:
        n1, n2 = N1, N2

        # Make sure that we start at the same heights
        if heights[n1] > heights[n2]:
            n1 = get_kth_ancestor(n1, heights[n1] - heights[n2])
        if heights[n1] < heights[n2]:
            n2 = get_kth_ancestor(n2, heights[n2] - heights[n1])

        # If n1 had been the ancestor of n2
        if n1 == n2:
            lca = n1
        else:
            # Run a `logrithmic` search, starting at max power reducing it
            # If match do nothing, otherwise update n1, n2 to where it didn't match
            for power in range(MAX_POWER - 1, -1, -1):
                if ancestors[n1][power] != ancestors[n2][power]:
                    n1, n2 = ancestors[n1][power], ancestors[n2][power]
            lca = ancestors[n1][0]

        return  heights[N1] - heights[lca], heights[N2] - heights[lca]

    def get_kth_ancestor(n: int, k: int) -> int:
        while k > 0 and n != -1:
            jump = int(math.log2(k))
            n, k = ancestors[n][jump], k - (1 << jump)

        return n

    # Max width of ancestors
    MAX_POWER = math.ceil(math.log2(N)) + 1
    ancestors: list[list[int]] = [[-1] * MAX_POWER for i in range(N + 1)]
    heights: dict[int, int] = {}

    # Convert edges into adjacency list
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n1].append(n2)
        adjl[n2].append(n1)

    # Do a DFS traversal, compute the ancestors and the depth
    stack: list[tuple[int, int, int]] = [(1, 0, 0)]
    while stack:
        curr, prev, height = stack.pop()
        ancestors[curr][0] = prev
        heights[curr] = height
        for next_ in adjl[curr]:
            if next_ != prev:
                stack.append((next_, curr, height + 1))

    # Compute the ancestors for powers up to MAX_N
    for power in range(1, MAX_POWER):
        for node in range(1, N + 1):
            ancestors[node][power] = ancestors[ancestors[node][power - 1]][power - 1]

    # Compute the results
    results: list[int] = []
    for n1, n2, E in queries:
        L, R = get_distances_from_lca(n1, n2)
        if E <= L:
            results.append(get_kth_ancestor(n1, E))
        elif E < L + R:
            results.append(get_kth_ancestor(n2, L + R - E))
        else:
            results.append(n2)

    return results

# Testing the solution
assert sloth_naptime(5, [(4,2), (1,4), (5,4), (3,4)], 6, [(3,5,2), (3,5,4), (1,5,5), (4,5,4), (1,5,4), (3,5,0)]) == [5, 5, 5, 5, 5, 3]
assert sloth_naptime(3, [(3,2), (2,1)], 3, [(2,2,2), (1,1,2), (3,3,3)]) == [2,1,3]
assert sloth_naptime(21, [(6, 9), (8, 1), (11, 10), (18, 17), (5, 17), (14, 7), (11, 14), (4, 18), (17, 13), (1, 14), (16, 21), (19, 6), (19, 14), (5, 9), (14, 16), (3, 20), (19, 15), (7, 2), (17, 3), (12, 7)], 21, [(15, 12, 16), (14, 10, 8), (17, 2, 16), (6, 14, 5), (4, 9, 5), (16, 16, 18), (18, 18, 12), (10, 8, 10), (7, 4, 3), (3, 10, 11), (11, 12, 4), (4, 20, 3), (13, 16, 17), (6, 2, 1), (17, 6, 13), (1, 9, 14), (9, 11, 9), (12, 6, 1), (14, 14, 19), (12, 12, 6), (16, 13, 4)]) == [12, 10, 2, 14, 9, 16, 18, 8, 6, 10, 12, 3, 16, 19, 6, 9, 11, 7, 14, 12, 9]

In [29]:
# Correct but memory limit exceeded
def cycle_free_flow(N: int, E: int, edges: list[tuple[int, int, int]], Q: int, queries: list[tuple[int, int]]) -> list[int]:
    def get_min_flow(N1: int, N2: int) -> int:
        n1, n2 = N1, N2
        flow: float = math.inf
        if heights[n1] > heights[n2]:
            n1, flow = get_kth_ancestor(n1, heights[n1] - heights[n2])
        if heights[n2] > heights[n1]:
            n2, flow = get_kth_ancestor(n2, heights[n2] - heights[n1])

        if n1 == n2:
            return int(flow)

        # Run a 'binary search'
        for power in range(MAX_POWER - 1, -1, -1):
            (n1_ancestor, n1_flow), (n2_ancestor, n2_flow) = ancestors[n1][power], ancestors[n2][power]
            if n1_ancestor != n2_ancestor:
                n1, n2, flow = n1_ancestor, n2_ancestor, min(n1_flow, n2_flow)

        return int(min(flow, ancestors[n1][0][1], ancestors[n2][0][1]))

    def get_kth_ancestor(curr: int, K: int) -> tuple[int, float]:
        flow: float = math.inf
        while curr != -1 and K > 0:
            jump = int(math.log2(K))
            curr, next_flow = ancestors[curr][jump]
            flow = min(flow, next_flow)
            K = K - (1 << jump)

        return curr, flow

    # Maximum power for computing the ancestors
    MAX_POWER = int(math.log2(N)) + 1
    ancestors: list[list[tuple[int, int]]] = [[(-1, 0)] * MAX_POWER for i in range(N + 1)]
    heights: dict[int, int] = dict()

    # Convert edges into Adjl list
    adjl: collections.defaultdict[int, list[tuple[int, int]]] = collections.defaultdict(list)
    for n1, n2, w in edges:
        adjl[n1].append((n2, w))
        adjl[n2].append((n1, w))

    # Run DFS and compute the level 1 ancestors
    stack: list[tuple[int, int]] = [(1, 0)]
    ancestors[1][0] = (0, 0)
    while stack:
        curr, height = stack.pop()
        for next_node, next_weight in adjl[curr]:
            if next_node not in heights:
                heights[next_node] = height + 1
                ancestors[next_node][0] = (curr, next_weight)
                stack.append((next_node, height + 1))

    # Compute the ancestors
    for power in range(1, MAX_POWER):
        for node in range(1, N + 1):
            prev_ancestor, prev_flow = ancestors[node][power - 1]
            next_ancestor, next_flow = ancestors[prev_ancestor][power - 1]
            ancestors[node][power] = next_ancestor, min(prev_flow, next_flow)

    # Process each query
    results: list[int] = []
    for n1, n2 in queries:
        results.append(get_min_flow(n1, n2))

    return results

# Testing the solution
assert cycle_free_flow(10, 9, [(7,10,1680), (10,5,18380), (5,8,8111), (8,9,22597), (3,10,27859), (4,3,9944), (6,10,26721), (1,10,4444), (2,5,2282)], 10, [(10,7), (4,2), (5,4), (9,6), (5,3), (4,10), (4,5), (2,1), (6,9), (9,4)]) == [1680, 2282, 9944, 8111, 18380, 9944, 9944, 2282, 8111, 8111]
assert cycle_free_flow(3, 3, [(3,2,4814), (2,1,1832)], 3, [(2,1), (1,2), (3,1)]) == [1832, 1832, 1832]
assert cycle_free_flow(5, 4, [(4,2,10348), (1,4,2690), (5,4,9807), (3,4,8008)], 5, [(5,4), (1,5)]) == [9807, 2690]
assert cycle_free_flow(5, 4, [(1,3,2653), (4,1,322), (5,1,8657), (2,4,4896)], 5, [(4,2), (2,5), (1,3), (4,5)]) == [4896, 322, 2653, 322]

In [30]:
def sortPeople(names: list[str], heights: list[int]) -> list[str]:
    return list(map(lambda x: x[1], sorted(zip(heights, names), reverse=True)))

# Testing the solution
assert sortPeople(["Mary","John","Emma"], [180,165,170]) == ["Mary","Emma","John"]
assert sortPeople(["Alice","Bob","Bob"], [155,185,150]) == ["Bob","Alice","Bob"]

In [31]:
def frequencySort(nums: list[int]) -> list[int]:
    freq: collections.Counter = collections.Counter(nums)
    result: list[int] = []
    for k, v in sorted(freq.most_common(), key=lambda x: (x[1], -x[0])):
        for i in range(v):
            result.append(k)

    return result

# Testing the solution
assert frequencySort([1,1,2,2,2,3]) == [3,1,1,2,2,2]
assert frequencySort([2,3,1,3,2]) == [1,3,3,2,2]

Codeforces Practice - LCA

In [32]:
def tree_queries_better(N: int, Q: int, edges: list[tuple[int, int]], queries: list[tuple[int, list[int]]]) -> list[bool]:
    """
    Thought process:
    For each query figure out the deepest node
    Iterate through each node in current query - `node`:
        - Init curr as deepest
        - Get kth ancestor such that curr and node are at same level
        - Either curr == node or node in adjl[ancestor[curr][0]]
    """

    # Init variables
    MAX_POWER = int(math.log2(N)) + 1
    ancestors: list[list[int]] = [[0 for j in range(MAX_POWER)] for i in range(N + 1)]

    @functools.cache
    def get_kth_ancestor(N: int, K: int) -> int:
        while N != 0 and K > 0:
            jump = int(math.log2(K))
            N, K = ancestors[N][jump], K - (1 << jump)

        return N

    # Create adjacency list
    adjl: collections.defaultdict[int, set[int]] = collections.defaultdict(set)
    for n1, n2 in edges:
        adjl[n1].add(n2)
        adjl[n2].add(n1)

    # Do a DFS to find level 1 ancestor and the height
    heights: dict[int, int] = dict()
    stack: list[tuple[int, int, int]] = [(1, 0, 0)]
    while stack:
        curr, prev, height = stack.pop()
        heights[curr] = height
        ancestors[curr][0] = prev
        for next_node in adjl[curr]:
            if next_node not in heights:
                stack.append((next_node, curr, height + 1))

    # Compute the ancestors
    for power in range(1, MAX_POWER):
        for node in range(1, N + 1):
            ancestors[node][power] = ancestors[ancestors[node][power - 1]][power - 1]

    # Process the queries
    results: list[bool] = []
    for _, query in queries:
        # Check the deepest node
        deepest_node: int = 1
        for n in query:
            if heights[n] > heights[deepest_node]:
                deepest_node = n

        # Process again through each node
        for n in query:
            curr = get_kth_ancestor(deepest_node, heights[deepest_node] - heights[n])
            if curr != n and n not in adjl[ancestors[curr][0]]:
                results.append(False)
                break
        else:
            results.append(True)

    return results

# Testing the solution
assert tree_queries_better(10, 6, [(1,2), (1,3), (1,4), (2,5), (2,6), (3,7), (7,8), (7,9), (9,10)], [(4, [3,8,9,10]), (3, [2,4,6]), (3, [2,1,5]), (3, [4,8,2]), (2, [6,10]), (3, [5,4,7])]) == [True, True, True, True, False, False]

In [33]:
def tree_queries(N: int, Q: int, edges: list[tuple[int, int]], queries: list[tuple[int, list[int]]]) -> list[bool]:
    # Initialize variables
    TIME: int = 0
    TIN: list[int] = [-1 for i in range(N + 1)]
    TOUT: list[int] = [-1 for i in range(N + 1)]
    AH: dict[int, tuple[int, int]] = dict()

    # Edges to Adj list
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n1].append(n2)
        adjl[n2].append(n1)

    # DFS traversal to keep track of in, out times; the ancestor and height
    stack: list[tuple[int, int, int, int]] = [(1, 1, 0, 0)]
    while stack:
        curr, prev, idx, height = stack[-1]
        AH[curr] = prev, height

        # If adding for the first time
        if idx == 0:
            TIN[curr] = TIME
            TIME += 1

        # If all its children have been visited
        if idx == len(adjl[curr]):
            stack.pop()
            TOUT[curr] = TIME
            TIME += 1

        # Children pending visit
        else:
            stack[-1] = (curr, prev, idx + 1, height)
            if prev != adjl[curr][idx]:
                stack.append((adjl[curr][idx], curr, 0, height + 1))

    # Solve for each query
    results: list[bool] = []
    for q, query in queries:

        # Find the deepest node
        deepest_node: int = 1
        for n in query:
            if AH[n][1] > AH[deepest_node][1]:
                deepest_node = n

        # Check if each node is ancestor of deepest_node
        result = all(map(lambda x: TIN[AH[x][0]] <= TIN[deepest_node] and  TOUT[deepest_node] <= TOUT[AH[x][0]], query))
        results.append(result)

    return results

# Testing the solution
assert tree_queries(10, 6, [(1,2), (1,3), (1,4), (2,5), (2,6), (3,7), (7,8), (7,9), (9,10)], [(4, [3,8,9,10]), (3, [2,4,6]), (3, [2,1,5]), (3, [4,8,2]), (2, [6,10]), (3, [5,4,7])]) == [True, True, True, True, False, False]

In [34]:
def one_tree_queries(N: int, edges: list[tuple[int, int]], Q: int, queries: list[tuple[int, int, int, int, int]]) -> list[bool]:
    """
    Per question, we can travel any node/edge any number of times.
    Assume path exists from s-e, we can introduce arbitrary 2Z (Z: 1 .. inf).
    Since it is 2Z, parity (odd / even) would remain same. So it suffices to check if
    we have a length of matching parity as k that is less than or equal to k

    Given query: h1, h2, s, e, k
    Paths possible:
        1. Direct path: s - e
        2. Indirect path via helpers: s - h1 - h2 - e
        3. Indirect path via helpers: s - h2 - h1 - e
    """

    def get_kth_ancestor(curr: int, K: int) -> int:
        while curr != -1 and K > 0:
            jump = int(math.log2(K))
            curr, K = ancestors[curr][jump], K - (1 << jump)
        return curr

    def compute_distance(N1: int, N2: int) -> int:
        # Compute LCA; dist from (N1 - LCA) + (N2 - LCA)
        n1, n2 = N1, N2
        if heights[n1] > heights[n2]:
            n1 = get_kth_ancestor(n1, heights[n1] - heights[n2])
        if heights[n2] > heights[n1]:
            n2 = get_kth_ancestor(n2, heights[n2] - heights[n1])

        if n1 == n2:
            lca = n1
        else:
            # `log` search to find LCA
            for power in range(MAX_POWER - 1, -1, -1):
                if ancestors[n1][power] != ancestors[n2][power]:
                    n1, n2 = ancestors[n1][power], ancestors[n2][power]
            lca = ancestors[n1][0]

        return abs(heights[lca] - heights[N1]) + abs(heights[lca] - heights[N2])

    # Initialize variables
    MAX_POWER = int(math.log2(N)) + 1
    ancestors: list[list[int]] = [[-1] * MAX_POWER for i in range(N + 1)]
    heights: dict[int, int] = {}
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)

    # Convert edges into adjl
    for n1, n2 in edges:
        adjl[n1].append(n2)
        adjl[n2].append(n1)

    # DFS to compute the ancestors
    stack: list[tuple[int, int, int]] = [(1, 0, 0)]
    while stack:
        curr, prev, height = stack.pop()
        ancestors[curr][0] = prev
        heights[curr] = height
        for next_node in adjl[curr]:
            if next_node not in heights:
                stack.append((next_node, curr, height + 1))

    # Compute ancestors
    for power in range(1, MAX_POWER):
        for node in range(1, N + 1):
            if ancestors[node][power - 1] != -1:
                ancestors[node][power] = ancestors[ancestors[node][power - 1]][power - 1]

    results: list[bool] = []
    for h1, h2, s, e, k in queries:
        # Precompute distances
        SE, SH1, SH2, H1E, H2E = compute_distance(s, e), compute_distance(s, h1), compute_distance(s, h2), compute_distance(h1, e), compute_distance(h2, e)
        SH1H2E = SH1 + 1 + H2E
        SH2H1E = SH2 + 1 + H1E
        # Direct path
        if SE % 2 == k % 2 and SE <= k:
            results.append(True)
        # Indirect path, via helper1
        elif SH1H2E % 2 == k % 2 and SH1H2E <= k:
            results.append(True)
        # Indirect path, via helper2
        elif SH2H1E % 2 == k % 2 and SH2H1E <= k:
            results.append(True)
        else:
            results.append(False)

    return results

# Testing the solution
assert one_tree_queries(5, [(1,2), (2,3), (3,4), (4,5)], 5, [(1,3,1,1,1), (1,3,1,2,2), (1,4,1,3,2), (1,4,1,3,3), (4,2,3,3,9), (5,2,3,3,9)]) == [False, True, True, False, True, False]
assert one_tree_queries(3, [(1,2), (2,3)], 2, [(1,3,1,1,1), (1,3,1,1,2), (1,3,1,1,3), (1,3,1,1,4),(3,1,1,1,1)]) == [False, True, True, True, False]

In [35]:
def blood_cousins(N: int, parents: list[int], Q: int, queries: list[tuple[int, int]]) -> list[int]:
    """
    Note that for this problem, there can be multiple components

    For each query (v, p)
    1. Find the pth ancestor of v: u (Using LCA)
    2. Compute the number of nodes of subtree with `u` as root at depth[v] (Using TIN, TOUT times and binary search)

    On second part, store the tin, tout times and store the tin times into a vector depth wise.
    Count of nodes of subtree 'u' and depth 'd' is UB_index(depth[v] - tout[u]) - LB_index(depth[v], tin[u])
    """

    def get_kth_ancestor(curr: int, K: int) -> int:
        while curr != 0 and K > 0:
            jump = int(math.log2(K))
            curr, K = ancestors[curr][jump], K - (1 << jump)
        return curr

    # Initialize variables
    MAX_POWER = int(math.log2(N)) + 1
    TIME: int = 0
    ancestors: list[list[int]] = [[0] * MAX_POWER for i in range(N + 1)]
    TIN: list[int] = [-1 for i in range(N + 1)]
    TOUT: list[int] = [-1 for i in range(N + 1)]
    heights: dict[int, int] = {}
    depth_map: collections.defaultdict[int, list] = collections.defaultdict(list)
    adjl: collections.defaultdict[int, list] = collections.defaultdict(list)

    # Parents to Adj List
    for i, p in enumerate(parents, start=1):
        adjl[p].append(i)

    # DFS search to compute ancestors, tin, tout times and the heights
    stack: list[tuple[int, int, int, int]] = [(i, 0, 0, 0) for i in adjl[0][::-1]]
    while stack:
        curr, prev, height, idx = stack[-1]
        if idx == 0:
            TIN[curr] = TIME
            depth_map[height].append(TIME)
            heights[curr] = height
            ancestors[curr][0] = prev
            TIME += 1
        if idx == len(adjl[curr]):
            stack.pop()
            TOUT[curr] = TIME
            TIME += 1
        else:
            stack[-1] = curr, prev, height, idx + 1
            stack.append((adjl[curr][idx], curr, height + 1, 0))

    # Compute ancestors
    for power in range(1, MAX_POWER):
        for node in range(1, N + 1):
            if ancestors[node][power - 1] != -1:
                ancestors[node][power] = ancestors[ancestors[node][power - 1]][power - 1]

    # Process the queries
    results: list[int] = []
    for V, P in queries:
        U = get_kth_ancestor(V, P)
        if U == 0:
            results.append(0)
        else:
            UB = bisect.bisect_right(depth_map[heights[V]], TOUT[U])
            LB = bisect.bisect_left(depth_map[heights[V]], TIN[U])
            results.append(UB - LB - 1)

    return results

# Testing the solution
assert blood_cousins(6, [0,1,1,0,4,4], 7, [(1,1), (1,2), (2,1), (2,2), (4,1), (5,1), (6,1)]) == [0, 0, 1, 0, 0, 1, 1]

In [36]:
def two_buttons(N: int, M: int, steps: int = 0) -> int:
    if M == N:
        return steps
    elif M < N:
        return steps + N - M
    else:
        return two_buttons(N, M // 2, steps + 1) if M % 2 == 0 else two_buttons(N, (M + 1) // 2, steps + 2)

# Testing the solution
assert two_buttons(7, 12) == 2
assert two_buttons(9, 1) == 8
assert two_buttons(1, 9) == 7

In [37]:
def party(N: int, supervisor: list[int]) -> int:
    # Convert edges into adjl
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for emp, boss in enumerate(supervisor, start=1):
        adjl[boss].append(emp)

    # Compute the max height
    max_height = 0
    stack: list[tuple[int, int]] = [(i, 1) for i in adjl[-1]]
    while stack:
        curr, height = stack.pop()
        max_height = max(max_height, height)
        for next_node in adjl[curr]:
            stack.append((next_node, height + 1))

    return max_height

# Testing the solution
assert party(5, [-1, 1, 2, 1, -1]) == 3

In [38]:
def kefa_and_park(N: int, K: int, cats: list[int], edges: list[tuple[int, int]]) -> int:
    # Convert edges into adjacency list
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n1].append(n2)
        adjl[n2].append(n1)

    # Do a DFS traversal until we reach the leaf node
    # If along a path we have more than K consequtive cats, break
    count = 0
    stack: list[tuple[int, int, int]] = [(1, 0, 0)]
    while stack:
        curr, prev, cat_count = stack.pop()
        is_leaf, safe_path = True, cats[curr - 1] + cat_count <= K
        if safe_path:
            for next_node in adjl[curr]:
                if next_node != prev:
                    is_leaf = False
                    stack.append((next_node, curr, cat_count + 1 if cats[curr - 1] else 0))

        count += int(is_leaf and safe_path)

    return count

# Testing the solution
assert kefa_and_park(7, 1, [1,0,1,1,0,0,0], [(1,2), (1,3), (2,4), (2,5), (3,6), (3,7)]) == 2
assert kefa_and_park(4, 1, [1,1,0,0], [(1,2), (1,3), (1,4)]) == 2

In [39]:
def cyclic_components(N: int, E: int, edges: list[tuple[int, int]]) -> int:
    """
    The take away: Component is a cycle iff all the vertices has degree of 2
    """

    # Convert edges into adjl
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n1].append(n2)
        adjl[n2].append(n1)

    # Do a DFS traversal and color components, ensure all vertices have degree 2
    color = cycles = 0
    visited: set[int] = set()
    for node in range(1, N + 1):
        if node not in visited:
            is_cyclic = True
            stack: list[int] = [node]
            while stack:
                curr = stack.pop()
                visited.add(curr)
                is_cyclic = is_cyclic and len(adjl[curr]) == 2
                for next_node in adjl[curr]:
                    if next_node not in visited:
                        stack.append(next_node)

                        # Adding this one simple line improves runtime greatly
                        # Until we visit that node itself, we would continue adding
                        # that node into our stack that would greatly impact our overall runtime
                        visited.add(next_node)

            cycles += int(is_cyclic)

    return cycles

# Testing the solution
assert cyclic_components(5, 4, [(1,2), (3,4), (5,4), (3,5)]) == 1
assert cyclic_components(4, 4, [(1,2), (2,3), (1,3), (1,4)]) == 0
assert cyclic_components(17, 15, [(1,8), (1,12), (5,11), (11,9), (9,15), (15,5), (4,13), (3,13), (4,3), (10,16), (7,10), (16,7), (14,3), (14,4), (17,6)]) == 2

In [40]:
def graph_without_long_directed_paths(N: int, E: int, edges: list[tuple[int, int]]) -> str:
    """
    The end graph must have nodes that all have either inward pointing or outward pointing.

    We can color the nodes as inward pointing or outward pointing, ensure that when we
    revist a node in a cycle the color is still valid.

    Once colors are determined, go through the edges once again and compute the direction of the edge.
    """

    # Convert edges into Adjl
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n1].append(n2)
        adjl[n2].append(n1)

    # DFS traversal to color nodes
    colors: dict[int, int] = dict()
    stack: list[tuple[int, bool]] = [(1, True)]
    while stack:
        curr, flag = stack.pop()
        colors[curr] = flag
        for next_node in adjl[curr]:
            if next_node not in colors:
                stack.append((next_node, not flag))
                colors[next_node] = not flag
            elif colors[next_node] == flag:
                return ''

    # Determine the direction of the edges
    directions: list[str] = []
    for n1, n2 in edges:
        directions.append('0' if colors[n1] else '1')

    return ''.join(directions)

# Testing the solution
assert graph_without_long_directed_paths(6, 5, [(1,5), (2,1), (1,4), (3,1), (6,1)]) in ('10100', '01011')
assert graph_without_long_directed_paths(5, 5, [(1,2), (2,3), (3,4), (4,5), (5,1)]) == ''
assert graph_without_long_directed_paths(4, 4, [(1,2), (2,3), (3,4), (4,1)]) in ('1010', '0101')

Matrix Exponentation

In [41]:
def plant(N: int) -> int:
    MOD = 10**9 + 7
    factor = pow(2, N - 1, MOD)
    return factor * (3 + 2 * (factor - 1))

# Testing the solution
assert plant(8) == 32896
assert plant(2) == 10

In [42]:
def magic_gems_dp(N: int, M: int) -> int:
    MOD = 10**9 + 7
    dp: collections.defaultdict[int, int] = collections.defaultdict(lambda: 0)
    dp[0] = 1
    for i in range(1, N + 1):
        dp[i] = (dp[i - 1] + dp[i - M]) % MOD

    return dp[N]

# Testing the solution
assert magic_gems_dp(4, 2) == 5
assert magic_gems_dp(3, 2) == 3

In [43]:
def magic_gems_equation_solving(N: int, M: int) -> int:
    def count_arrangements(x: int, y: int) -> int:
        return math.factorial(x + y) // (math.factorial(x) * math.factorial(y))

    MOD = 10**9 + 7
    count = 0
    for i in range(N // M + 1):
        j = N - i * M
        count = (count + count_arrangements(i, j)) % MOD

    return count

# Testing the solution
for i in range(1, 100):
    for j in range(2, 100):
        assert magic_gems_dp(i, j) == magic_gems_equation_solving(i, j)

In [44]:
def magic_gems(N: int, M: int) -> int:
    """
    Based on dp recurance relation: dp[n] = dp[n - 1] + dp[n - M].
    This would take linear time complexity but can be optimized using matrix exponentation.

    Transformation matrix T repr initial state, N >= M (since N < M the initial state is simply [1 .... ])
    It can help us get from f(n) to f(n + 1)
    T . f(n) => f(n + 1)

    Starting at T = f(1), we can get to f(n) by doing T ** (N - M + 1)
    T ** (N - M + 1) instead of N since we already started at point N >= M

    Assume F(5, 3):
        f(1) = T would repr F(3, 3)
        f(2) = T . T = F(4, 3)
        f(3) = T . T . T = F(5, 3)

    Which is T ** (N - M + 1)
    """

    def multiply_matrix(A: list[list[int]], B: list[list[int]], mod: int) -> list[list[int]]:
        # (m x n) . (n x p) => (m x p)
        assert len(A[0]) == len(B)
        M, N, P = len(A), len(A[0]), len(B[0])
        result: list[list[int]] = [[0 for j in range(P)] for i in range(M)]
        for i in range(M):
            for j in range(P):
                for k in range(N):
                    result[i][j] = (result[i][j] + A[i][k] * B[k][j]) % mod
        return result

    def power_of_matrix(base: list[list[int]], exp: int, mod):
        size = len(base)
        # Identity matrix
        result: list[list[int]] = [[1 if i == j else 0 for j in range(size)] for i in range(size)]
        # Power of matrix by squaring
        while exp > 0:
            if exp % 2 == 1:
                result = multiply_matrix(result, base, mod)
            base = multiply_matrix(base, base, mod)
            exp //= 2
        return result

    MOD = 10**9 + 7

    if N < M:
        return 1

    else:

        # Setup transformation matrix
        T = [[0 for j in range(M)] for i in range(M)]
        T[0][0] = T[0][M - 1] = 1
        for i in range(1, M):
            T[i][i - 1] = 1

        # Raise T to the power (N - M + 1)
        result_matrix = power_of_matrix(T, N - M + 1, MOD)

        # Return sum of T[0]
        return sum(result_matrix[0][i] % MOD for i in range(M)) % MOD

# Testing the solution
for i in range(1, 25):
    for j in range(2, 25):
        assert magic_gems_dp(i, j) == magic_gems(i, j)

LC Weekly - 28th July 2024

In [45]:
def canAliceWin(nums: list[int]) -> bool:
    sds = dds = 0
    for n in nums:
        if n >= 10:
            dds += n
        else:
            sds += n

    return sds != dds

# Testing the solution
assert canAliceWin([1,2,3,4,10]) == False
assert canAliceWin([1,2,3,4,5,14]) == True
assert canAliceWin([5,5,5,25]) == True

In [46]:
def nonSpecialCount(L: int, R: int) -> int:
    """
    A number is special iff it is a square of a prime number

    1. Generate all primes till sqrt(R).
    2. Iterate through list of primes if p * p is within L and R, we have one special number counted.
    """

    def SOE(N: int) -> list[int]:
        "Generate list of primes using Seive of erastosthenes"
        primes: list[int] = [True for i in range(N + 1)]
        primes[0] = primes[1] = False
        for i in range(2, N + 1):
            if primes[i]:
                primes[2*i::i] = [False] * len(primes[2*i::i])
        return [i for i in range(2, N + 1) if primes[i]]

    primes = SOE(int(math.sqrt(R)))
    special_count = 0
    for p in primes:
        if L <= p * p <= R:
            special_count += 1

    return R - L + 1 - special_count

# Testing the solution
assert nonSpecialCount(5, 7) == 3
assert nonSpecialCount(4, 16) == 11

CP Practice: Once again - 582B

In [47]:
def LNDS(N: int, T: int, nums: list[int]) -> int:

    # Store the maximum non decreasing subseq
    subseq: list[int] = []

    # Iterate N times or T times whichever is smaller
    # Ensure the loop runs only once if t is large enough to
    # make further processing redundant
    while N and T:
        initial: int = len(subseq)
        for n in nums:
            idx = bisect.bisect_right(subseq, n)
            if idx == len(subseq):
                subseq.append(n)
            else:
                subseq[idx] = n

        # diff represents how much the LIS can potentially grow in one iteration.
        diff = len(subseq) - initial

        N, T = N - 1, T - 1

    # length of the longest non-decreasing subsequence found after the first complet
    # pass through the array, plus the additional length contributed by the repetitions
    return len(subseq) + diff * T

# Testing the solution
assert LNDS(4, 3, [3,1,4,2]) == 5
assert LNDS(3, 2, [1,2,3]) == 4
assert LNDS(3, 2, [1,3,2]) == 3

LC POTD: https://leetcode.com/problems/minimum-deletions-to-make-string-balanced

In [48]:
def minimumDeletions(s: str) -> int:
    N = len(s)

    # Count a's to the right of idx
    count = 0
    a_counts: list[int] = [0 for i in range(N)]
    for i in range(N - 1, -1, -1):
        a_counts[i] = count
        if s[i] == 'a':
            count += 1

    # Count b's to the left of idx
    count = 0
    b_counts: list[int] = [0 for i in range(N)]
    for i in range(N):
        b_counts[i] = count
        if s[i] == 'b':
            count += 1

    left, right = 0, N - 1
    min_deletions, deletions = N, 0
    while left <= right:
        while left < right and s[left] == 'a':
            left += 1
        while left < right and s[right] == 'b':
            right -= 1

        min_deletions = min(min_deletions, deletions + a_counts[left] - deletions // 2, deletions + b_counts[right] - deletions // 2)
        left, right, deletions = left + 1, right - 1, deletions + 2

    return min_deletions

# Testing the solution
assert minimumDeletions("ababaaaabbbbbaaababbbbbbaaabbaababbabbbbaabbbbaabbabbabaabbbababaa") == 25
assert minimumDeletions("aababbab") == 2
assert minimumDeletions("bbaaaaabb") == 2
assert minimumDeletions("aabbb") ==  minimumDeletions("aaaaa") == minimumDeletions("bbb") == 0
assert minimumDeletions("a") == 0

Codeforces: Edu Round 168, Jul 30

In [49]:
def strong_password(s: str) -> str:
    password: list[str] = []
    replaced = False
    for ch in s:
        if password and password[-1] == ch and not replaced:
            password.append('a' if ch != 'a' else 'b')
            replaced = True
        password.append(ch)

    if not replaced:
        password.append('a' if password[-1] != 'a' else 'b')

    return ''.join(password)

# Testing the solution
assert strong_password("a") == "ab"
assert strong_password("aa") == "aba"
assert strong_password("passw") == "pasasw"
assert strong_password("aba") == "abab"

In [50]:
def make_three_regions(N: int, grid: list[str]) -> int:
    counts = 0
    for i in range(1, N - 1):
        # Count for pattern in 1st row
        if grid[0][i - 1: i + 2] == "..." and grid[1][i - 1: i + 2] == "x.x":
            counts += 1
        # Count for pattern in 2nd row
        if grid[1][i - 1: i + 2] == "..." and grid[0][i - 1: i + 2] == "x.x":
            counts += 1

    return counts

# Testing the solution
assert make_three_regions(8, [".......x", ".x.xx..."]) == 1
assert make_three_regions(2, ["..", ".."]) == 0
assert make_three_regions(3, ["xxx", "xxx"]) == 0
assert make_three_regions(9, ["..x.x.x.x", "x.......x"]) == 2

In [51]:
def even_positions(n: int, brackets: str) -> int:
    "Greedy Approach"
    cost = 0
    stack: list[int] = []
    for i in range(n):
        if brackets[i] == '(':
            stack.append(i)
        elif brackets[i] == ')' or stack:
            cost += i - stack.pop()
        else:
            stack.append(i)

    return cost

# Testing the solution
assert even_positions(6, "_(_)_)") == 5
assert even_positions(2, "_)") == 1
assert even_positions(8, "_)_)_)_)") == 4
assert even_positions(8, "_(_)_(_)") == 8

#### Upsolving codeforces contest from yesterday

In [52]:
def maximize_root_better(N: int, values: list[int], parents: list[int]) -> int:
    """
    Let us do a binary search.

    Check if we can add delta of Min - 0, Maximum - max(values).

    We can add delta to a node if all subtrees can provide the value of delta. If for any
    node the value is lesser than delta, it could still be possible as long as its
    children have value equal to delta + delta_diff.
    Accumulate delta_diff until leaf node.

    Time: O (N log N) - TLE in python
    """
    def is_valid(target: int) -> bool:
        if target <= values[0]:
            return True
        elif not adjl[0]:
            return False
        else:
            stack: list[tuple[int, int]] = [(i, target - values[0]) for i in adjl[0]]
            while stack:
                curr, curr_target = stack.pop()
                if not adjl[curr] and curr_target > values[curr]:
                    return False
                else:
                    diff = max(curr_target - values[curr], 0)
                    for next_node in adjl[curr]:
                        stack.append((next_node, target + diff))
            return True

    # Create the tree
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for node, parent in enumerate(parents, start=1):
        adjl[parent - 1].append(node)

    # Do a binary search
    low, high = values[0], values[0] + max(values)
    while low <= high:
        mid = (low + high) // 2
        if is_valid(mid):
            low = mid + 1
        else:
            high = mid - 1

    return high

# Testing the solution
assert maximize_root_better(5, [2,5,3,9,6], [3,1,5,2]) == 6
assert maximize_root_better(4, [0,1,0,2], [1,1,3]) == 1
assert maximize_root_better(2, [3], [0]) == 3

In [53]:
def maximize_root(N: int, values: list[int], parents: list[int]) -> int:
    # Convert parents to adj list
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for node, parent in enumerate(parents, start=1):
        adjl[parent - 1].append(node)

    # Do a BFS traversal to get the leaf nodes first, granted that the leaf
    # nodes might be visited later in time when we start poping,
    # neverthless if a node had a child, the child would be popped first
    queue: collections.deque[int] = collections.deque([0])
    order: list[int] = []
    while queue:
        curr = queue.popleft()
        order.append(curr)
        for next_node in adjl[curr]:
            queue.append(next_node)

    # Traverse in reverse BFS order and calculate the maximum value it can have as a root
    dp: dict[int, int] = dict()
    while order:
        curr = order.pop()
        if not adjl[curr]:
            dp[curr] = values[curr]
        else:
            min_ = min(dp[next_node] for next_node in adjl[curr])
            dp[curr] = min(min_, (min_ + values[curr]) // 2)

    # Find the maximum value that root node can have
    return values[0] + (min(dp[next_node] for next_node in adjl[0]) if adjl[0] else 0)

# Testing the solution
assert maximize_root(4, [0,1,0,2], [1,1,3]) == 1
assert maximize_root(5, [2,5,3,9,6], [3,1,5,2]) == 6
assert maximize_root(2, [3], [0]) == 3

In [54]:
def decoding_genome_memo_brute(N: int, M: int, k: int, forbidden: set[str]) -> int:
    @functools.cache
    def backtrack(curr_ord: int, remaining: int) -> int:
        if remaining == 0:
            return 1
        else:
            counts = 0
            for i in range(M):
                if curr_ord == -1 or chr(ord('a') + curr_ord) + chr(ord('a') + i) not in forbidden:
                    counts += backtrack(i, remaining - 1)
            return counts

    return backtrack(-1, N)

# Testing the solution
assert decoding_genome_memo_brute(3, 3, 2,  {"ab", "ba"}) == 17
assert decoding_genome_memo_brute(3, 3, 0, set()) == 27
assert decoding_genome_memo_brute(2, 1, 1, {"aa"}) == 0

In [55]:
def decoding_genome_brute(N: int, M: int, k: int, forbidden: set[str]) -> int:
    "Time: O(N * M * M), Space: O(M)"
    dp: list[int] = [1 for i in range(-1, M)]
    for remaining in range(1, N + 1):
        next_dp: list[int] = [0 for i in range(-1, M)]
        for curr_ord in range(M - 1, -2, -1):
            for next_ord in range(M):
                if curr_ord == -1 or chr(ord('a') + curr_ord) + chr(ord('a') + next_ord) not in forbidden:
                    next_dp[curr_ord + 1] += dp[next_ord + 1]
        dp = next_dp

    return dp[0]

# Testing the solution
assert decoding_genome_brute(3, 3, 2,  {"ab", "ba"}) == 17
assert decoding_genome_brute(3, 3, 0, set()) == 27
assert decoding_genome_brute(2, 1, 1, {"aa"}) == 0

In [56]:
def decoding_genome(N: int, M: int, k: int, forbidden: set[str]) -> int:
    def ch2int(ch: str) -> int:
        ord_ = ord(ch)
        if ord('a') <= ord_ <= ord('z'):
            return ord_ - ord('a')
        else:
            return ord_ - ord('A') + 26

    def matrix_multiply(A: list[list[int]], B: list[list[int]], mod: int) -> list[list[int]]:
        # (M x N) . (P x Q) => (M x Q)
        M, N, P, Q = len(A), len(A[0]), len(B), len(B[0])
        assert N == P, f"Matrices of invalid dimensions provided, got: {(M, N), (P, Q)}"
        result: list[list[int]] = [[0 for j in range(Q)] for i in range(M)]

        for i in range(M):
            for j in range(Q):
                for k in range(N):
                    result[i][j] = (result[i][j] + A[i][k] * B[k][j]) % mod

        return result

    def matrix_power(base: list[list[int]], power: int, mod: int) -> list[list[int]]:
        N = len(matrix)
        result: list[list[int]] = [[1 if i == j else 0 for j in range(N)] for i in range(N)]

        while power:
            if power & 1:
                result = matrix_multiply(result, base, mod)
            base = matrix_multiply(base, base, mod)
            power >>= 1

        return result

    # We need answer modulo 1e9 + 7
    MOD = 10**9 + 7

    # Get the transition matrix
    matrix: list[list[int]] = [[1 for j in range(M)] for i in range(M)]
    for pair in forbidden:
        ord1, ord2 = ch2int(pair[0]), ch2int(pair[1])
        matrix[ord1][ord2] = 0

    # Raise tran matrix to power N - 1
    result_matrix = matrix_power(matrix, N - 1, MOD)

    # Result is sum of all cells
    return sum(sum(row) % MOD for row in result_matrix) % MOD

# Testing the solution
assert decoding_genome(3, 3, 2,  {"ab", "ba"}) == 17
assert decoding_genome(3, 3, 0, set()) == 27
assert decoding_genome(2, 1, 1, {"aa"}) == 0

In [57]:
def random_mood(N: int, P: float) -> float:
    switch, no_switch = P, 1 - P
    happy, sad = 1., 0.
    while N:
        if N & 1:
            happy, sad = (happy * no_switch + sad * switch), (happy * switch + sad * no_switch)
        switch, no_switch = switch * no_switch + no_switch * switch, switch * switch + no_switch * no_switch
        N >>= 1

    return happy

# Testing the solution
assert math.isclose(random_mood(11, 0.06), 0.6225404294, abs_tol=1e-6)
assert math.isclose(random_mood(2, 0.1), 0.82, abs_tol=1e-6)
assert math.isclose(random_mood(1, 0.7), 0.3, abs_tol=1e-6)

In [58]:
def string_mood_brute(N: int) -> int:
    MOD = 10 ** 9 + 7
    sad, happy = 0, 1
    for length in range(N):
        sad_, happy_ = 0, 0
        for ord_ in range(26):
            ch = chr(ord('A') + ord_)
            if ch == 'H':
                sad_ = (sad_ + happy) % MOD
                happy_ = (happy_ + happy) % MOD
            elif ch in ('S', 'D'):
                sad_ = (sad_ + sad) % MOD
                happy_ = (happy_ + sad) % MOD
            elif ch in ('A', 'E', 'I', 'O', 'U'):
                sad_ = (sad_ + happy) % MOD
                happy_ = (happy_ + sad) % MOD
            else:
                sad_ = (sad_ + sad) % MOD
                happy_ = (happy_ + happy) % MOD
        sad, happy = sad_, happy_

    return happy

# Testing the solution
assert string_mood_brute(1) == 19
assert string_mood_brute(2) == 403
assert string_mood_brute(11) == 145418665

In [59]:
def string_mood_better(N: int) -> int:
    MOD = 10 ** 9 + 7

    sad, happy = 0, 1
    for length in range(N):
        sad_ = (6 * happy) + (20 * sad)
        happy_ = (7 * sad) + (19 * happy)
        sad, happy = sad_, happy_

    return happy % MOD

# Testing the solution
assert string_mood_better(1) == 19
assert string_mood_better(2) == 403
assert string_mood_better(11) == 145418665

In [60]:
def string_mood(N: int) -> int:
    def matrix_multiply(A: list[list[int]], B: list[list[int]], mod: int) -> list[list[int]]:
        M, N, P, Q = len(A), len(A[0]), len(B), len(B[0])
        assert N == P, "Matrix dimensions are incorrect"
        result: list[list[int]] = [[0 for j in range(Q)] for i in range(M)]
        for i in range(M):
            for j in range(Q):
                for k in range(N):
                    result[i][j] = (result[i][j] + A[i][k] * B[k][j]) % mod

        return result

    def matrix_power(base: list[list[int]], exp: int, mod: int) -> list[list[int]]:
        N = len(base)
        result: list[list[int]] = [[1 if i == j else 0 for j in range(N)] for i in range(N)]
        while exp:
            if exp & 1:
                result = matrix_multiply(result, base, mod)
            base = matrix_multiply(base, base, mod)
            exp >>= 1

        return result

    # Sad, Happy
    base: list[list[int]] = [[20, 6], [7, 19]]
    result: list[list[int]] = matrix_power(base, N, 10 ** 9 + 7)

    return result[1][1]

# Testing the solution
assert string_mood(1) == 19
assert string_mood(2) == 403
assert string_mood(11) == 145418665

In [61]:
def fibo(N: int) -> int:
    def matrix_multiply(A: list[list[int]], B: list[list[int]], mod: int) -> list[list[int]]:
        M, N, P, Q = len(A), len(A[0]), len(B), len(B[0])
        assert N == P, "matrix dimensions are incorrect"
        result: list[list[int]] = [[0 for j in range(Q)] for i in range(M)]
        for i in range(M):
            for j in range(Q):
                for k in range(N):
                    result[i][j] = (result[i][j] + A[i][k] * B[k][j]) % mod

        return result

    def matrix_exp(base: list[list[int]], exp: int, mod: int) -> list[list[int]]:
        N = len(base)
        result: list[list[int]] = [[1 if i == j else 0 for j in range(N)] for i in range(N)]
        while exp:
            if exp & 1:
                result = matrix_multiply(result, base, mod)
            base = matrix_multiply(base, base, mod)
            exp >>= 1

        return result

    base: list[list[int]] = [[0, 1], [1, 1]]
    result: list[list[int]] = matrix_exp(base, N, 998244353)
    return result[1][0]

# Testing the solution
assert fibo(0) == 0
assert fibo(1234) == 4936310
assert fibo(345639696828452375) == 213237811
assert fibo(419384601238473729475639183948326177846782649592628790267300203877) == 389871463